In [1]:
import os
import pandas as pd
import glob
import cv2

In [2]:
# information about the videos that are to be masked - use datasheet on Drive - csv file
seq_info = r"G:\.shortcut-targets-by-id\11AE4tEztgQ4SLqv4eoIUjjTJQvPkhRNy\MuscaChase\Musca-Musca_Chase_AnimalChart_experimentNotes.csv"
temp = pd.read_csv(seq_info)
videos = temp[(temp.BodyMarked.notnull())]
videos

,Filename,Date of experiment,Framerate,No. of intact males in the box,No. of ablated males in the box,No. of females in the box,Marking on the ablated males,Marking on the females,Date of collection of flies,Cameras used,...,Digitizer,Digitization status,BodyMarked,"start, stop on DLTdv","start frame number, cine video","end frame number, cine video",difference,DURATION OF CHASE,Caught?,Unnamed: 24
81,2023-04-19_Trial1_Set1_5k_16024.cine,2023-04-19,5000,10,10,5,Horizontal Line,Dot,2023-04-19,16024_20654_20655,...,NaN,NaN,"Yes, Tanvi","70, 786",already chasing when filming began,-10362.0,#VALUE!,#VALUE!,yes (II),NaN


In [31]:
cam1_path = r"D:\Musca Chase\2023-04-19_Musca chase_Highspeed\2023-04-19_Trial1_Set1_5k_16024.avi"
cam2_path = r"D:\Musca Chase\2023-04-19_Musca chase_Highspeed\2023-04-19_Trial1_Set1_5k_20654.avi"
cam3_path = r"D:\Musca Chase\2023-04-19_Musca chase_Highspeed\2023-04-19_Trial1_Set1_5k_20655.avi"
camlist = [cam1_path, cam2_path, cam3_path]

DLTxypoints_path = r"G:\.shortcut-targets-by-id\11AE4tEztgQ4SLqv4eoIUjjTJQvPkhRNy\MuscaChase\MS_Analysis\MuscaChasingMusca\preprocessing_image\\2023_04_19-Trial1_Set1_5kxypts.csv"

In [4]:
temp_dir = r"../../../MuscaChase/dataFolders/MuscaChasingMusca/Preprocessing_Images/temp/"

In [19]:
total_frames

-1

In [32]:
cap = cv2.VideoCapture(camlist[0])

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_frames

-1

In [33]:
frame_width

1280

In [34]:
cap.isOpened()

True

In [12]:
ii = 1
for cam in camlist[0:1]:
    # read the video
    cap = cv2.VideoCapture(cam)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    #get the output video name
    outfilename = cam.split("\\")[-1][:-4]
    # Define the codec and output video
    out = cv2.VideoWriter(outfilename + '.mp4',
                          cv2.VideoWriter_fourcc(*'mp4v'),
                          fps,
                          (frame_width, frame_height))
    
    # get the frame start and stop in the video
    Frames = videos.loc[videos.Filename == [name for name in videos.Filename if cam.split("\\")[-1][:-9] in name], 'start, stop on DLTdv']
    start = int(Frames.values[0].split(',')[0])
    stop = int(Frames.values[0].split(',')[1])
    # Specify range
    start_frame = start
    end_frame = stop
    
    # get the coordinates for transparent rectangle
    coords = pd.read_csv(DLTxypoints_path)
    subset = coords.loc[start_frame:end_frame, :]
    
    cam_subset = subset.loc[:,[columns for columns in subset.columns if str(ii) in columns]] 
    chaser_x, chaser_y =  cam_subset.iloc[:, 0:2]
    chasee_x, chasee_y = cam_subset.iloc[:, 2:4]
    
    # Set the starting point
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    current_frame = start_frame

    while cap.isOpened() and current_frame < end_frame:
        ret, frame = cap.read()
        if not ret:
            print('oops')
            break
        
        #frame number
        ff = 0
        print(f'frame num {ff}')
        
        # Create black background
        black_frame = np.zeros_like(frame)
        
#         # Parameters for transparent rectangle
#         x, y = chaser_x[ff] - 100, chaser_y[ff] - 100    # Top-left corner of transparent area
#         dt = 500            # Width and height of the transparent rectangle

#         # Copy only the transparent rectangle from original frame
#         black_frame[y:y+dt, x:x+dt] = frame[y:y+dt, x:x+dt]

#         # Write the masked frame
# #         out.write(black_frame)
#         # Save frame as image
#         frame_filename = os.path.join(temp_dir, f"frame_{saved_frame_count:05d}.png")
#         cv2.imwrite(frame_filename, black_frame)
#         ff+=1
#     cap.release()
#     out.release()
#     cv2.destroyAllWindows()

oops


In [14]:
ret, frame = cap.read()

In [15]:
ret

False

In [27]:
import cv2
print(cv2.getBuildInformation())


General configuration for OpenCV 4.6.0 =====================================
  Version control:               unknown

  Extra modules:
    Location (extra):            C:/b/abs_d8ltn27ay8/croot/opencv-suite_1676452046667/work/opencv_contrib-4.6.0/modules
    Version control (extra):     unknown

  Platform:
    Timestamp:                   2023-02-15T12:02:59Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.22.1
    CMake generator:             Ninja
    CMake build tool:            C:/b/abs_d8ltn27ay8/croot/opencv-suite_1676452046667/_build_env/Library/bin/ninja.exe
    MSVC:                        1916
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
   